# Machine Learning 

### Homework 1

In [17]:
import numpy as np
import matplotlib as mpl

**Decision Tree:**

Input data:

In [149]:
data = [[0,0,0,1,0,1,0],
    [0,1,0,0,1,1,1],
    [1,0,1,0,1,0,0],
    [0,0,1,1,0,0,1],
    [0,0,1,1,0,0,0]]

ds = np.array(ds)
ds

array([[0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1],
       [1, 0, 1, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0]])

ValueError: all the input arrays must have same number of dimensions

$$ P(y=0) = /frac{5}{7} $$

$$ P(y=1) = /frac{2}{7} $$

In [32]:
P_y0 = 5.0/7
P_y1 = 2.0/7

Entropy:

In [80]:
def entropy(labels):
    labels = labels.tolist()
    prob_dict = {x:labels.count(x)/len(labels) for x in labels}
    probs = np.array(list(prob_dict.values()))

    return - probs.dot(np.log2(probs))

In [91]:
X

array([[0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 1],
       [1, 0, 0, 1],
       [0, 1, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 1]])

In [141]:
split_indexes = np.where(X[:,3]==1)[0]

left = []
right = []

for i, row in enumerate(X):
    if i in split_indexes:
        left.append(row)
    else:
        right.append(row)
        
left = np.array(left)
right = np.array(right)


(array([[0, 0, 1, 1],
        [1, 0, 0, 1],
        [0, 1, 0, 1]]), array([[0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 1, 0],
        [1, 1, 0, 0]]))

In [138]:
a = []
a.append([1,2,3,4])
a.append([1,2,3,5])

[[1, 2, 3, 4], [1, 2, 3, 4, 6, 7]]

In [139]:
np.array(a)

array([[1, 2, 3, 4],
       [1, 2, 3, 5]])

In [116]:
2 in split_indexes

True

In [109]:
np.split(X, split_indexes, axis=0)

[array([[0, 0, 1, 0],
        [0, 1, 0, 0]]), array([[0, 0, 1, 1]]), array([[1, 0, 0, 1],
        [0, 1, 1, 0],
        [1, 1, 0, 0]]), array([[0, 1, 0, 1]])]

In [85]:
entropy(Y)

0.863120568566631

In [ ]:
Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])

Information Gain:

In [ ]:
def info_gain(data,split_attr):
  
    total_entropy = entropy(Y)
    
    #Calculate the values and the corresponding counts for the split feature
    vals,counts= np.unique(X[:,split_feature],return_counts=True)
    
    # calculate new entropy for this split
    new_entropy
    
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(new_entropy) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [ ]:
info_gain(X,Y, 4)

In [88]:
X[:,3]

array([0, 0, 1, 1, 0, 0, 1])

In [90]:
vals,counts= np.unique(X[:,3],return_counts=True)
vals, counts

(array([0, 1]), array([4, 3], dtype=int64))

In [ ]:
for each feature:
    calculate entropy

In [13]:
class Node(object):
    def __init__(self, value, children=None, parent=None):
        self.value = value
        self.parent = parent
        self.children = children or []

    def add_child(self, value):
        new_child = Node(value, parent=self)
        self.children.append(new_child)
        return new_child
    
    def is_root(self):
        return self.parent is None

    def is_leaf(self):
        return not self.children

    def __str__(self):
        if self.is_leaf():
            return str(self.value)
        return '{value} [{children}]'.format(value=self.value, children=', '.join(map(str, self.children)))

ID3 (Examples, Target_Attribute, Attributes)
    Create a root node for the tree
    If all examples are positive, Return the single-node tree Root, with label = +.
    If all examples are negative, Return the single-node tree Root, with label = -.
    If number of predicting attributes is empty, then Return the single node tree Root,
    with label = most common value of the target attribute in the examples.
    Otherwise Begin
        A ← The Attribute that best classifies examples.
        Decision Tree attribute for Root = A.
        For each possible value, vi, of A,
            Add a new tree branch below Root, corresponding to the test A = vi.
            Let Examples(vi) be the subset of examples that have the value vi for A
            If Examples(vi) is empty
                Then below this new branch add a leaf node with label = most common target value in the examples
            Else below this new branch add the subtree ID3 (Examples(vi), Target_Attribute, Attributes – {A})
    End
    Return Root

In [150]:
def id3(data):
    
    labels = data[:][-1]
    
    if np.mean(labels) == 1:
        pass
    elif np.mean(labels) == 0:
        pass
    else:
        # find best attribute
    
    
    
    

SyntaxError: invalid syntax (<ipython-input-150-18d95cd00fc8>, line 4)

In [11]:
n = Node(12)
a = n.add_child(23)

In [12]:
print(n)

12 [23]
